**Author:** Beatrice Occhiena s314971. See [`LICENSE`](https://github.com/beatrice-occhiena/Computational_intelligence/blob/main/LICENSE) for details.
- institutional email: `S314971@studenti.polito.it`
- personal email: `beatrice.occhiena@live.it`
- github repository: [https://github.com/beatrice-occhiena/Computational_intelligence.git](https://github.com/beatrice-occhiena/Computational_intelligence.git)

**Resources:** These notes are the result of additional research and analysis of the lecture material presented by Professor Giovanni Squillero for the Computational Intelligence course during the academic year 2023-2024 @ Politecnico di Torino. They are intended to be my attempt to make a personal contribution and to rework the topics covered in the following resources.
- [https://github.com/squillero/computational-intelligence](https://github.com/squillero/computational-intelligence)
- Stuart Russel, Peter Norvig, *Artificial Intelligence: A Modern Approach* [3th edition]

.

.


# Problem Solving by Searching
---

## Why search?

> Search techniques are utilized in problem-solving scenarios when there is a clear, predefined goal that needs to be achieved, and the problem can be logically broken down into different states and actions.

Search algorithms are essential for tackling problems in various domains and are especially useful in cases when:
- The problem's **complexity** is beyond straightforward mathematical modeling or doesn't have a known analytical solution.
- We **lack prior knowledge** of the exact sequence of actions leading to a solution.
- We intend to systematically explore a **large solution space**, requiring an examination of various paths to find the best one.
- We seek **optimal or near-optimal solutions** in extensive search spaces.

### Problem solving agent
A problem-solving agent is a type of goal-based agent capable of determining a sequence of actions leading to a desired goal. This agent achieves this by exploring the problem's state space, which encompasses all potential states reachable from the initial state through a sequence of actions.

The agent's program comprises two key components:
- **Problem Formulation** This entails the process of deciding which actions and states to consider based on the specified goal. 🗺️..
- **Search Algorithm** This involves determining the next action to take given the current state in order to progress toward the goal. 🧭..

### Conditions for solving problems by searching
While path search algorithms can be applied to a wide range of problems, there are some conditions and considerations that make them more suitable for certain types of problems.

When the task environment is
- discrete
- fully observable
- deterministic
- static
- completelly known

> In this case, the agent can be said to have `full control` of the environment, therefore it can plan ahead and choose a sequence of actions that will lead to the desired goal. The process of finding a sequence of actions is called *search*.

## Problem solving agents
1. Initial state
2. Set of actions
3. Transition model
4. Goal test function
5. Path cost function


- state space
- path -> solution
- search tree vs search graph: 
  => treat states and actions as **atomic** entities

## Search strategies
Their performance is measured in terms of:
- Completeness: is the algorithm guaranteed to find a solution when there is one?
- Optimality: does the strategy find the best solution?
- Time complexity: how long does it take to find a solution?
- Space complexity: how much memory is needed to perform the search?
Complexity depends on:
- b: maximum branching factor of the search tree
- d: depth of the least-cost solution

### Uninformed search strategies
- Breadth-first search
- Uniform-cost search
- Depth-first search
- Depth-limited search
- Iterative deepening search
- Bidirectional search
- Beam search

### Informed search strategies
- Greedy best-first search
- A* search
- Recursive best-first search (RBFS)
- Simplified Memory-bounded A* search (SMA*)



